In [ ]:
import numpy as np
import os
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout, Input, Conv2DTranspose, Cropping2D
import keras.utils.all_utils as kr_utils
import keras.regularizers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [ ]:

np.random.seed = 24
imgdir_train = '../input/traindata/images'
maskdir_train = '../input/traindata/masks'
imgdir_test = '../input/testdata/images'
maskdir_test = '../input/testdata/masks'
imgs = []
masks = []
names = []

def extractxy(imgdir, maskdir, showimgs = False):

    imgs = []
    masks = []
    names = []
    
    for filename in os.listdir(imgdir):
        names.append(filename)
        imgpath = os.path.join(imgdir, filename)
        maskpath = os.path.join(maskdir, filename)
        imginst = cv2.imread(imgpath)
        maskinst = cv2.imread(maskpath)

        imginst = cv2.cvtColor(imginst, cv2.COLOR_BGR2RGB)
        imgind = np.nonzero(maskinst > 0)
        maskinst[imgind[0],imgind[1],:] = 1

        imgs.append(imginst)
        masks.append(maskinst)
        
    masks = np.array(masks)
    masks = masks[:,:,:,0]
    masks = np.expand_dims(masks,3)
    imgs = np.array(imgs)
    if showimgs:
        for i in range(masks.shape[0]):
            fig, axs = plt.subplots(1,2)
            #print(names[i])
            axs[0].imshow(masks[i])
            axs[0].set_title(names[i])
            axs[1].imshow(imgs[i])
            axs[1].set_title(names[i])
            plt.plot()
    
    return imgs, masks

#scaler = MinMaxScaler()
X_train, y_train = extractxy(imgdir_train, maskdir_train)
X_train = X_train / 255
#scaler.fit_transform(X_train[])
X_test, y_test = extractxy(imgdir_test, maskdir_test, True)
X_test = X_test / 255
#scaler.transform(X_test)

In [ ]:
drop_ratio = 8/100
max_conv_maps = 256

def encoders(mod_input, conv_maps, filter_sz, padding_type):
    ei = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(mod_input)
    ei = Dropout(drop_ratio)(ei)
    ei = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(ei)
    return ei

def decoders(enc_input, up_input, up_maps, up_sz, up_stride, padding_type, conv_maps, filter_sz, Print = False):
    up_do = Conv2DTranspose(up_maps, up_sz, strides=up_stride, padding = padding_type)(up_input)
    if (Print):
        print(up_do.shape)
    up_do = Concatenate()([enc_input,up_do])
    up_do = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(up_do)
    up_do = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(up_do)
    return up_do

cnn_input = Input(shape=(360, 480, 3))
eo1 = encoders(cnn_input, max_conv_maps / (2 ** 4), (3,3), 'same')
#crp1 = Cropping2D(((4,4),(0,0)))(eo1)
#print(eo1.shape)
#print(crp1.shape)
ei2 = MaxPooling2D((2,2))(eo1)

eo2 = encoders(ei2, max_conv_maps / (2 ** 3), (3,3), 'same')
#crp2 = Cropping2D(((2,2),(0,0)))(eo2)
#print(eo2.shape)
#print(crp2.shape)
ei3 = MaxPooling2D((2,2))(eo2)

eo3 = encoders(ei3, max_conv_maps / (2 ** 2), (3,3), 'same')
#crp3 = Cropping2D(((1,1),(0,0)))(eo3)
#print(eo3.shape)
#print(crp3.shape)
ei4 = MaxPooling2D((2,2))(eo3)

eo4 = encoders(ei4, max_conv_maps / (2 ** 1), (3,3), 'same')
#crp4 = Cropping2D(((1,0),(0,0)))(eo4)
#print(eo4.shape)
#print(crp4.shape)
ei5 = MaxPooling2D((3,3))(eo4)

peak = Conv2D(max_conv_maps / (2 ** 0),(3,3),padding = 'same', activation='relu')(ei5)
peak = Dropout(drop_ratio)(peak)
peak = Conv2D(max_conv_maps / (2 ** 0),(3,3),padding = 'same', activation='relu')(peak)

do4 = decoders(eo4, peak, max_conv_maps / (2 ** 1), (3,3), (3,3), 'same', 128, (3,3))
do3 = decoders(eo3, do4, max_conv_maps / (2 ** 2), (2,2), (2,2), 'same', 64, (3,3))
do2 = decoders(eo2, do3, max_conv_maps / (2 ** 3), (2,2), (2,2), 'same', 32, (3,3))
do1 = decoders(eo1, do2, max_conv_maps / (2 ** 4), (2,2), (2,2), 'same', 16, (3,3))

fin_out = Conv2D(1, (1,1), padding = 'same', activation='sigmoid')(do1)

model = Model(inputs = cnn_input, outputs = fin_out)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
checkpoint_callbk = tf.keras.callbacks.ModelCheckpoint(
    "best_model", # name of file to save the best model to
    monitor="val_accuracy", # prefix val to specify that we want the model with best macroF1 on the validation data
    verbose=1, # prints out when the model achieve a better epoch
    mode="max", # the monitored metric should be maximized
    save_freq="epoch", # clear
    save_best_only=True, # of course, if not, every time a new best is achieved will be savedf differently
    save_weights_only=True # this means that we don't have to save the architecture, if you change the architecture, you'll loose the old weights
)
kf = KFold(n_splits = 4, shuffle = True, random_state = 24)

for trainind, testind in kf.split(X_train,y_train):
    model.fit(x = X_train[trainind,:,:,:], y = y_train[trainind], validation_data = (X_train[testind,:,:,:], y_train[testind]), callbacks=[checkpoint_callbk], epochs=10,
    batch_size=5, verbose=0)
model.load_weights("best_model")

In [ ]:
model.evaluate(X_test, y_test, verbose = 1)
pred = model.predict(X_test)
pred[pred > (1/2)] = 255
pred[pred <= (1/2)] = 0
print(np.unique(y_test))
pic_ious = []
pic_fdr = []
pic_precision = []
pic_recall = []
iou_zeros = 0
zero_pos = 0
for samp in zip(pred[:,:,:,0], y_test[:,:,:,0]):
    ind_pred = np.nonzero(samp[0] > 0)
    indx_pred = ind_pred[0]
    indy_pred = ind_pred[1]
    ind_pred = np.column_stack((indx_pred,indy_pred))
    pred_pos = ind_pred.shape[0]
    
    ind_test = np.nonzero(samp[1] > 0)
    indx_test = ind_test[0]
    indy_test = ind_test[1]
    ind_test = np.column_stack((indx_test,indy_test))
    pos = ind_test.shape[0]
    
    coord_all = np.concatenate((ind_test,ind_pred), axis = 0)
    ind_test = set([tuple(ele) for ele in ind_test])
    ind_pred = set([tuple(ele) for ele in ind_test])
    coord_unique = set(tuple(row) for row in coord_all)
    intersect = np.array([val for val in ind_test & ind_pred]) 
    true_pos = intersect.shape[0]
    false_pos = pred_pos - true_pos
    
    if (pos) != 0:
        recall = true_pos / (pos) # false discorvery rate
    else:
        recall = 0
    
    pic_recall.append(recall)
    
    if (false_pos + true_pos) != 0:
        precision = true_pos / (false_pos + true_pos) # false discorvery rate
    else:
        precision = 0
        
    pic_precision.append(precision)
    # false_pos / (pos)
    if (false_pos + true_pos) != 0:
        fdr = false_pos / (false_pos + true_pos) # false discorvery rate
    else:
        zero_pos += 0
        fdr = 0
        
    pic_fdr.append(fdr)
        
    if len(coord_unique) > 0:
        iou = (true_pos) / (len(coord_unique))
    else:
        iou_zeros += 1
        iou = 0
    pic_ious.append(iou)
    #print(len(coord_unique))
    #print(intersect.shape[0])
    #print(f'IOU = {iou}')
    #print('-------------------')

pic_fdr = np.array(pic_fdr)
print(f'Mean FDR = {np.mean(pic_fdr)}')
print(f'Proportion of zero positives = {zero_pos / pic_fdr.shape[0]}')
    
pic_ious = np.array(pic_ious)
print(f'Mean IOU = {np.mean(pic_ious)}')
print(f'Proportion of intersection zeros = {iou_zeros / pic_ious.shape[0]}')
print(pic_recall)
print(pic_precision)
plt.scatter(pic_recall,pic_precision)
#fig, axs = plt.subplots(1,1)
#axs.plot(recall,precision)
'''
fig, axs = plt.subplots(1,3)
imgnum = 17
#pred[imgnum,20:90,20:90,0] = 255 
ind = np.nonzero(pred[imgnum,:,:,0] > 0)
temparr = np.zeros((360,480,1))
temparr[ind[0],ind[1],:] = 255
axs[0].imshow(temparr)
axs[1].imshow(y_test[imgnum])
axs[2].imshow(X_test[imgnum])
#360 by 480
temparr = np.reshape(temparr,(360,480))
#dcluster = KMeans(1).fit(temparr)
dcluster = DBSCAN(min_samples = 200).fit(temparr)
print(np.unique(dcluster.labels_))
clusterind = np.nonzero(dcluster.labels_ == 0)[0]
A = ind[0][clusterind]
B = ind[1][clusterind]
temparr2 = np.zeros((360,480,1))
temparr2[A,B,:] = 255
axs[2].imshow(temparr2)
plt.plot()
'''

In [ ]:
from sklearn.metrics import jaccard_score
y_true = np.array([[0, 1, 1],[1, 1, 0]])
y_pred = np.array([[0, 1, 1],[1, 0, 0]])

dum1 = np.array([1,2,3,4])
dum2 = np.array([1,2,3,4])
dum3 = np.column_stack((dum1,dum2))
print(dum3)

arr1 = np.concatenate((y_true,y_pred), axis = 0)
print(arr1)

uniqarr = np.array(list(set(tuple(row) for row in arr1)))